In [1]:
%pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import sys
sys.path.insert(0, '../src/')
import pandas as pd
import spacy
from collections import defaultdict
# from data_cleaning import split_df, df_lang
from data_transformation import get_X_y, distribution
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Perceptron
from sklearn.preprocessing import OneHotEncoder


## 1. Data Collection

We begin by scrapping Wiktionary.org for *feminine*, *masculine*, and *neuter* nouns in *Polish*, *German*, *Spanish*, and *French*.


In [3]:
# run webscrapper

## 2. Data Cleaning

The raw data in json format must be cleaned: removing nouns with *spaces*, *hyphens*, *numbers*, *abbreviations*, *initials* and finally those that are *proper nouns*.

### 2.1 Read data

In [4]:
# read json file and load it as a DataFrame
path = '../data/raw_scraped_data.json'
raw_df = pd.read_json(path)
raw_df

,noun,gender,lang
0,a,masculine,Polish
1,Aalborg,masculine,Polish
2,aalen,masculine,Polish
3,Aarhus,masculine,Polish
4,abacysta,masculine,Polish
...,...,...,...
328184,zurrona,feminine,Spanish
328185,zutana,feminine,Spanish
328186,zwingliana,feminine,Spanish
328187,zwingliana,feminine,Spanish


### 2.2 Initial filter

In [5]:
# remove numbers, hypens, spaces, and periods
no_nums = raw_df[(~raw_df['noun'].str.contains('-| |\.|1|2|3|4|5|6|7|8|9|0'))]
print(no_nums.shape)
# remove full uppercase
no_caps = no_nums[(~no_nums['noun'].str.isupper())]
print(no_caps.shape)


(301257, 3)
(299192, 3)


### 2.3 Further cleaning

Break df into each language to: 
- remove duplicates in each language
- remove nouns that begin with a capital letter (except for German) 

In [6]:
from typing import List, Tuple
from spacy.language import Language
from collections import namedtuple


def df_lang(df: pd.DataFrame)-> List[Tuple[str, pd.DataFrame]]:
    """
    Splits main df by 'lang' column, and creates new sub
    dataFrames

    returns:
        list: namedtuple (lang, df)
    """
    Sub_df = namedtuple('Sub_df', ['lang', 'df'])
    languages = df['lang'].unique()
    dataframes = [df[df['lang'] == lang] for lang in languages]
    return [Sub_df(lang, sub_df) for lang, sub_df in zip(languages, dataframes)]

In [7]:
spacy.load("de_core_news_sm")

# separate df for each language
dfs = df_lang(no_caps)

# remove nouns that begin with captial letters (except German)
temp = []
for sub_df in dfs:
    if sub_df.lang != 'German':
        temp.append(sub_df.df[~sub_df.df['noun'].str.istitle()])
    else: # otherwise, for German, pass into SpaCy and filter out Proper nouns
        words = pd.Series(sub_df.df['noun']).tolist()
        nlp = spacy.load("de_core_news_sm")
        text = " ".join(words) 
        nlp.max_length = len(text) 
        doc = nlp(text) 
        tokens = [token.text for token in doc if token.pos_ != 'PROPN']
        temp.append(sub_df.df[sub_df.df['noun'].isin(tokens)])
clean = pd.concat(temp)

In [8]:
clean

,noun,gender,lang
0,a,masculine,Polish
2,aalen,masculine,Polish
4,abacysta,masculine,Polish
5,abak,masculine,Polish
7,abakawir,masculine,Polish
...,...,...,...
328184,zurrona,feminine,Spanish
328185,zutana,feminine,Spanish
328186,zwingliana,feminine,Spanish
328187,zwingliana,feminine,Spanish


## 3 Transform and Encode data
#### 3.1 Create function to add # to all nouns
Make all nouns the same length, this is done by preprending '#' to nouns until they are a uniform length


In [9]:
# get length of longest noun
max_length = clean['noun'].str.len().max() # 38

def add_filler(word):
    """ preprends n amount #'s to a word 
    based on a max_length"""
    if len(word) < max_length: # if word len() is less than the max len()
        diff = max_length - len(word) # we subtract the current word len() by the max len(): diff
        return '#' * diff + word # we prepend n(diff) amount of '#'s to the word and then return it
    return word # if len() of word is NOT less then the max len, then just return it

# apply function to every value in column 'noun'
clean['noun'] = clean['noun'].apply(add_filler)
print(clean.head(5))

                                     noun     gender    lang
0  #####################################a  masculine  Polish
2  #################################aalen  masculine  Polish
4  ##############################abacysta  masculine  Polish
5  ##################################abak  masculine  Polish
7  ##############################abakawir  masculine  Polish


#### 3.2 
get a distribution of all languages and genders and get the lowest count

In [10]:
grouped = clean.groupby(['gender','lang']).size().unstack()
lowest_value = int(grouped.min().min())
print(distribution(clean))
print('lowest_value = ', lowest_value)


lang       French  German  Polish   Spanish
gender                                     
feminine   3801.0  3624.0  2555.0  116375.0
masculine  3911.0  2485.0  2681.0  119263.0
neuter        NaN  2392.0  4883.0       NaN
lowest_value =  2392


### 3.3 Reduce all language sub dfs and genders to a uniform amount (lowest_value)

In [11]:
reduced_df = clean.groupby(['lang', 'gender'])['noun', 'gender', 'lang'].sample(n=lowest_value) # reduce each language and gender by lowest_value
print(distribution(reduced_df))

/tmp/ipykernel_8297/1350974568.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  reduced_df = clean.groupby(['lang', 'gender'])['noun', 'gender', 'lang'].sample(n=lowest_value) # reduce each language and gender by lowest_value


lang       French  German  Polish  Spanish
gender                                    
feminine   2392.0  2392.0  2392.0   2392.0
masculine  2392.0  2392.0  2392.0   2392.0
neuter        NaN  2392.0  2392.0      NaN


#### 3.3 Define an encoding function
aim is to grab n amount of letters in a noun (starting from the end), and passing it into a one-hot encoder that will represent said letter(s) as a vector.
1. grab n amount of letters from each noun (this will be encoded)
2. intialize an encoder
3. encode n amount of letters
4. convert results into a dataframe (transformed df)
5. re index reduced df
6. return new dataframe which is made of : reduced df and transformed df

In [12]:
def encode(reduced_df, n=0):
    to_be_encoded = reduced_df['noun'].str[-n:] # grab n amount of letters start from the end to encode only
    ohe = OneHotEncoder(sparse_output=False) # initialize the encoder
    transformed = ohe.fit_transform(to_be_encoded.to_numpy().reshape(-1, 1)) # encode
    transformed_df = pd.DataFrame(transformed) # convert to a dataframe
    reduced_df.reset_index(inplace=True, drop=True) # reset indexes
    return pd.concat([reduced_df, transformed_df], axis=1) # create new dataframe of reduced df and transformed df

## 4 Train, test, fit, score

#### 4.1 create function to train,fit, test all languages with n amount of encoding



In [13]:
def multi_train_per_lang(df):
    results = defaultdict(lambda :defaultdict(list)) # to store scores from ML Models. dict->dict->list
    max_length = df['noun'].str.len().max() # get longest noun in whole dataset
    dfs = df_lang(df) # break df into smalled dfs based on language: spanish_df, french_df etc
    for sub_df in dfs: # for each language df
        for n in range(1, max_length + 1): # 
            encoded_df = encode(sub_df.df, n) # encode n amount of letters for the ith langauge df
            X, y = get_X_y(encoded_df) # X is the vector representationf for n amount of letters, y are the labels
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

            # now that we have a train test split, we can plug them into our ML models
            # KNN
            knn = KNeighborsClassifier(n_neighbors=3) # initialize a KNN class, 3 neighbors
            knn.fit(X_train, y_train) # train it 
            results['KNN'][n].append((knn.score(X_test, y_test), sub_df.lang)) # append score into results dict, along with name of sub df (French, German, etc)

            # Perceptron
            p = Perceptron(random_state=42) # initialize a Perceptron class, random state 42
            p.fit(X_train, y_train) # train it
            results['Perceptron'][n].append((p.score(X_test, y_test), sub_df.lang)) # append score to results dict, along with name of sub df (French, German, etc)

    return results # return s scores of l sub dfs in df, of n amount of letters encode, for c amount of ML models. Ex: results[c][n]: (s,l)

## 4.2 Experiment 1: train and test each language individually on n amount of letters encoded 
## takes ~5 mins to run!!

In [14]:
all_langs = multi_train_per_lang(reduced_df)
all_langs

defaultdict(<function __main__.multi_train_per_lang.<locals>.<lambda>()>,
            {'KNN': defaultdict(list,
                         {1: [(0.7262277951933125, 'French'),
                           (0.47214484679665736, 'German'),
                           (0.9380222841225627, 'Polish'),
                           (0.8307210031347962, 'Spanish')],
                          2: [(0.7878787878787878, 'French'),
                           (0.7395543175487466, 'German'),
                           (0.9568245125348189, 'Polish'),
                           (0.8401253918495298, 'Spanish')],
                          3: [(0.7931034482758621, 'French'),
                           (0.7862116991643454, 'German'),
                           (0.9394150417827298, 'Polish'),
                           (0.8756530825496343, 'Spanish')],
                          4: [(0.7105538140020898, 'French'),
                           (0.8036211699164345, 'German'),
                           (0.8725626740947

### 4.3 Show results for KNN

In [15]:
knn_results = all_langs['KNN']
knn_df = pd.DataFrame.from_dict(knn_results, orient="index", columns=['Polish', 'German', 'French', 'Spanish'])
knn_iterative_df = knn_df.applymap(lambda x: x[0]) # grab first element in the tuple (score, langauage)
knn_iterative_df.to_csv('../data/knn_per_language_results.csv') # save to csv
knn_iterative_df # display dataframe

,Polish,German,French,Spanish
1,0.726228,0.472145,0.938022,0.830721
2,0.787879,0.739554,0.956825,0.840125
3,0.793103,0.786212,0.939415,0.875653
4,0.710554,0.803621,0.872563,0.742947
5,0.712644,0.747911,0.786908,0.612330
6,0.655172,0.577298,0.639276,0.560084
7,0.560084,0.451253,0.564067,0.576803
8,0.550679,0.496518,0.486072,0.516196
9,0.477534,0.474234,0.548050,0.589342
10,0.543365,0.449164,0.519499,0.516196


### 4.4 Show results for Perceptron

In [16]:
perceptron_results = all_langs['Perceptron']
per_df = pd.DataFrame.from_dict(perceptron_results, orient="index", columns=['Polish', 'German', 'French', 'Spanish'])
per_iterative_df = per_df.applymap(lambda x: x[0]) # grab first element in the tuple (score, langauage)
per_iterative_df.to_csv('../data/perceptron_per_language_results.csv') # save as csv
per_iterative_df # display df


,Polish,German,French,Spanish
1,0.735632,0.651811,0.924095,0.806688
2,0.812957,0.736072,0.961003,0.826541
3,0.780564,0.800836,0.944290,0.867294
4,0.705329,0.794568,0.891365,0.747126
5,0.655172,0.746518,0.742340,0.684431
6,0.589342,0.639972,0.686630,0.637409
7,0.560084,0.552228,0.629526,0.655172
8,0.530825,0.539694,0.606546,0.592476
9,0.496343,0.479805,0.528552,0.589342
10,0.487983,0.490251,0.518802,0.590387


## 4.5 Expermiment 2: train on one lang, and test on a different langauage (shuffle)

In [17]:
def multi_train_per_lang_shuffle(df, max_length):
    results = defaultdict(lambda :defaultdict(lambda: defaultdict()))
    training_data = defaultdict()
    testing_data = defaultdict()
    trans_df = encode(df, max_length) # encode
    dfs = df_lang(trans_df)
    for sub_df in dfs: # for each language
        X, y = get_X_y(sub_df.df) # get X and y
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5) # split
        training_data[sub_df.lang] = (X_train, y_train) # add the x and y train for each language into a dict
        testing_data[sub_df.lang] = (X_test, y_test) # add the x and y testing for each language into a dict
        
    for i in training_data: # for every language training
        for j in testing_data: # for every language testing
       
            X_train__, y_train__ = training_data[i]
            X_test__, y_test__ = testing_data[j]


            knn = KNeighborsClassifier(n_neighbors=3) # initialize a KNN
            knn.fit(X_train__, y_train__) # fit it with training
            results['KNN'][i][j] = knn.score(X_test__, y_test__) # test it
 
            p = Perceptron(random_state=42) # initialize a Perceptron
            p.fit(X_train__, y_train__) # fit it with training
            results['Perceptron'][i][j] = p.score(X_test__, y_test__) # test it


    return results

### 4.6 Run experiment on only 4 letters encoded

In [18]:
shuffled = multi_train_per_lang_shuffle(reduced_df, 4)
shuffled


defaultdict(<function __main__.multi_train_per_lang_shuffle.<locals>.<lambda>()>,
            {'KNN': defaultdict(<function __main__.multi_train_per_lang_shuffle.<locals>.<lambda>.<locals>.<lambda>()>,
                         {'French': defaultdict(None,
                                      {'French': 0.8202341137123745,
                                       'German': 0.3879598662207358,
                                       'Polish': 0.34698996655518394,
                                       'Spanish': 0.4991638795986622}),
                          'German': defaultdict(None,
                                      {'French': 0.5861204013377926,
                                       'German': 0.7410813823857302,
                                       'Polish': 0.3481047937569677,
                                       'Spanish': 0.4991638795986622}),
                          'Polish': defaultdict(None,
                                      {'French': 0.49372909698996653,
       

### 4.7 show results of KNN

In [19]:
knn_shuffle_res = shuffled['KNN']
knn_shuffle_df = pd.DataFrame(knn_shuffle_res)
knn_shuffle_df.to_csv('../data/knn_res_lang_shuffle_4_encoding.csv') # save as csv
knn_shuffle_df

,French,German,Polish,Spanish
French,0.820234,0.586120,0.493729,0.490385
German,0.387960,0.741081,0.339465,0.352564
Polish,0.346990,0.348105,0.840580,0.340022
Spanish,0.499164,0.499164,0.541388,0.725334


### 4.8 Show results of Perceptron

In [20]:
percp_shuffle_res = shuffled['Perceptron']
percep_shuffle_df = pd.DataFrame(percp_shuffle_res)
percep_shuffle_df.to_csv('../data/percp_res_lang_shuffle_4_encoding.csv') # save as csv
percep_shuffle_df

,French,German,Polish,Spanish
French,0.633361,0.597826,0.492057,0.492057
German,0.346433,0.791806,0.339744,0.352843
Polish,0.335842,0.343367,0.858974,0.334448
Spanish,0.507525,0.501254,0.540970,0.714465


## 4.9 Experiment 3: Train and test on WHOLE dataset as one

In [21]:
encoded_df = encode(reduced_df, 4) #  4 letters encoded
X, y = get_X_y(encoded_df) # X is the vector representationf for n amount of letters, y are the labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# KNN
knn = KNeighborsClassifier(n_neighbors=3) # initialize a KNN class, 3 neighbors
knn.fit(X_train, y_train) # train it 
knn_score = knn.score(X_test, y_test) # get score

# Perceptron
p = Perceptron(random_state=42) # initialize a Perceptron class, random state 42
p.fit(X_train, y_train) # train it
p_score = p.score(X_test, y_test) # get score


### 4.10 display knn scores

In [22]:
knn_score

0.8346571906354515

### 4.11 display perceptron scores

In [23]:
p_score

0.7915969899665551